#### Prueba de Scraping

In [1]:
pip install selenium beautifulsoup4 pandas webdriver_manager pandas-gbq pyarrow xlrd openpyxl

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import random
import pandas as pd
from google.cloud import bigquery
import numpy as np
from google.oauth2 import service_account
import os
import glob
import re

### Web Scraping Ordenes de compra

In [3]:
def configurar_navegador():
    """Configura y devuelve el driver del navegador Chrome."""
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
    return driver

def escribir_texto_humanizado(elemento, texto):
    """Escribe texto en un elemento de forma humanizada, con pausas aleatorias."""
    elemento.clear()
    for caracter in texto:
        elemento.send_keys(caracter)
        pausa = random.uniform(0.1, 0.3)
        time.sleep(pausa)
        if random.random() < 0.1:  
            time.sleep(random.uniform(0.3, 0.7))

def hacer_clic_seguro(driver, xpath, mensaje_exito):
    """Intenta hacer clic en un elemento usando JavaScript para mayor fiabilidad."""
    try:
        elemento = driver.find_element(By.XPATH, xpath)
        time.sleep(random.uniform(0.3, 0.7))
        driver.execute_script("arguments[0].click();", elemento)
        print(mensaje_exito)
        return True
    except Exception as e:
        print(f"Error al hacer clic en {mensaje_exito}: {e}")
        return False

def iniciar_sesion(driver, usuario, clave):
    """Realiza el proceso de inicio de sesión."""
    input_usuario = driver.find_element(By.CSS_SELECTOR, 'input.p-inputtext.p-component.p-element.user-input')
    escribir_texto_humanizado(input_usuario, usuario)
    time.sleep(1)
    input_clave = driver.find_element(By.ID, 'textClave')
    escribir_texto_humanizado(input_clave, clave)
    time.sleep(random.uniform(0.8, 1.5))
    hacer_clic_seguro(driver, '//button[.//span[text()="Ingresar"]]', "Botón Ingresar clickeado")
    time.sleep(random.uniform(3, 5))

def navegar_a_servicios_transaccionales(driver):
    """Navega al menú de servicios transaccionales."""
    wait = WebDriverWait(driver, 20)
    boton_servicios = wait.until(EC.presence_of_element_located(
        (By.XPATH, '//button[.//span[text()="Tus servicios"]]')
    ))
    time.sleep(random.uniform(1.0, 2.5))
    boton_servicios.click()
    time.sleep(2)
    hacer_clic_seguro(
        driver, 
        '//a[@role="tab" and contains(.,"Servicios transaccionales")]',
        "Servicios transaccionales encontrado y clickeado"
    )
    time.sleep(3)

def navegar_a_gestion_orden_compra(driver):
    """Navega a la sección de Gestión de la Orden de Compra."""
    hacer_clic_seguro(
        driver, 
        '//a[@role="tab" and contains(.,"Gestión de la Orden de Compra")]',
        "Gestión de la Orden de Compra encontrado y clickeado"
    )
    time.sleep(3)

def navegar_a_consulta_orden_compra(driver):
    """Navega a la sección de Consulta de la Orden de Compra."""
    hacer_clic_seguro(
        driver, 
        '//a[@role="tab" and contains(.,"Consulta de la Orden de Compra")]',
        "Consulta de la Orden de Compra encontrado y clickeado"
    )
    time.sleep(5)

def ir_a_version_anterior(driver):
    """Navega a la versión anterior del sitio."""
    hacer_clic_seguro(
        driver, 
        '//button[.//span[text()="Ir a versión del sitio anterior"]]',
        "Botón 'Ir a versión del sitio anterior' encontrado y clickeado"
    )
    time.sleep(5)
    try:
        wait = WebDriverWait(driver, 10)
        boton_continuar = wait.until(EC.element_to_be_clickable(
            (By.XPATH, '//button[.//span[text()="Continuar al sitio anterior"]]')
        ))
        time.sleep(random.uniform(1.0, 2.0))
        driver.execute_script("arguments[0].click();", boton_continuar)
        print("Botón 'Continuar al sitio anterior' encontrado y clickeado")
    except Exception as e:
        print(f"Error al encontrar botón 'Continuar al sitio anterior': {e}")
    time.sleep(5)

def ir_a_pagina_siguiente(driver):
    """
    Hace clic en el botón de 'Siguiente' para navegar a la página siguiente.
    """
    try:
        print("Buscando el botón 'Siguiente'...")
        try:
            driver.switch_to.default_content()
            driver.switch_to.frame("main")
            print("Cambiado al iframe 'main' para buscar el botón Siguiente")
        except Exception as e:
            print(f"Error al cambiar al iframe: {e}")
        time.sleep(2)
        selectores = [
            "//img[contains(@src, '2_adelante.gif')]",
            "//img[@alt='Siguiente']",
            "//a[.//img[contains(@src, 'adelante')]]",
            "//a[.//img[contains(@src, 'siguiente')]]",
            "//input[@alt='Siguiente']"
        ]
        
        for selector in selectores:
            try:
                wait = WebDriverWait(driver, 5)
                boton = wait.until(EC.element_to_be_clickable((By.XPATH, selector)))
                time.sleep(random.uniform(0.8, 1.5))
                driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", boton)
                time.sleep(random.uniform(0.3, 0.7))
                if boton.tag_name.lower() == "img":
                    try:
                        enlace = boton.find_element(By.XPATH, "..")
                        if enlace.tag_name.lower() == "a":
                            boton = enlace
                            print("Usando el enlace padre de la imagen para el clic")
                    except:
                        pass
                driver.execute_script("arguments[0].click();", boton)
                print(f"Se ha hecho clic en el botón 'Siguiente' usando selector: {selector}")
                time.sleep(3)
                return True
            except Exception as e:
                print(f"No se pudo hacer clic en el botón con selector: {selector}")
        script = """
        var imagenes = document.querySelectorAll('img');
        for (var i = 0; i < imagenes.length; i++) {
            if (imagenes[i].src.includes('adelante.gif') || 
                imagenes[i].alt === 'Siguiente') {
                // Si la imagen está dentro de un enlace, hacer clic en el enlace
                if (imagenes[i].parentNode.tagName.toLowerCase() === 'a') {
                    imagenes[i].parentNode.click();
                    return "Enlace padre clickeado";
                }
                // De lo contrario, hacer clic en la imagen
                imagenes[i].click();
                return "Imagen clickeada";
            }
        }
        
        // Buscar también por el texto "Siguiente" en enlaces
        var enlaces = document.querySelectorAll('a');
        for (var j = 0; j < enlaces.length; j++) {
            if (enlaces[j].textContent.includes('Siguiente') || 
                enlaces[j].textContent.includes('siguiente') ||
                enlaces[j].href.includes('siguiente') ||
                enlaces[j].href.includes('next')) {
                enlaces[j].click();
                return "Enlace de texto clickeado";
            }
        }
        
        return false;
        """
        resultado = driver.execute_script(script)
        if resultado:
            print(f"Se ha hecho clic en el botón 'Siguiente' mediante JavaScript: {resultado}")
            time.sleep(3)
            return True
        try:
            script_paginacion = """
            var elementos = document.querySelectorAll('a, span, b');
            for (var i = 0; i < elementos.length; i++) {
                if (elementos[i].textContent.trim().match(/^\\d+$/) && 
                    (elementos[i].className.includes('active') || 
                     elementos[i].style.fontWeight === 'bold' ||
                     elementos[i].tagName.toLowerCase() === 'b')) {
                    // Encontramos el número de página actual
                    var paginaActual = parseInt(elementos[i].textContent.trim());
                    // Buscar el enlace a la siguiente página
                    for (var j = 0; j < elementos.length; j++) {
                        if (elementos[j].textContent.trim() == (paginaActual + 1)) {
                            elementos[j].click();
                            return "Número de página siguiente clickeado: " + (paginaActual + 1);
                        }
                    }
                }
            }
            return false;
            """
            resultado_paginacion = driver.execute_script(script_paginacion)
            if resultado_paginacion:
                print(f"Se ha hecho clic en el número de página siguiente: {resultado_paginacion}")
                time.sleep(3)
                return True
        except Exception as e:
            print(f"Error al intentar la paginación numérica: {e}")
        driver.save_screenshot("boton_siguiente.png")
        print("No se pudo encontrar el botón 'Siguiente'. Revisa la captura boton_siguiente.png")
        return False
        
    except Exception as e:
        print(f"Error al intentar ir a la página siguiente: {e}")
        driver.save_screenshot("error_pagina_siguiente.png")
        print("Se ha guardado una captura de pantalla para diagnóstico: error_pagina_siguiente.png")
        return False
    finally:
        try:
            driver.switch_to.default_content()
        except:
            pass

def seleccionar_checkboxes(driver, usar_seleccionar_todos=True):
    """
    Selecciona los checkboxes de las órdenes en la tabla imitando el comportamiento humano.
    Maneja el cambio a la nueva ventana y cambio a iframe.
    """
    try:
        cambiar_a_nueva_ventana(driver)
        cambiar_a_iframe_main(driver)
        #Aqui va la funcion de pagina siguiente
        ir_a_pagina_siguiente(driver)
        time.sleep(2)
        print("Buscando checkboxes en la tabla...")
        if usar_seleccionar_todos:
            cambiar_a_iframe_main(driver)
            seleccionar_checkbox_principal(driver)
        
        driver.switch_to.default_content()
        print("Volviendo al contexto principal del documento")
        
    except Exception as e:
        print(f"Error al seleccionar checkboxes: {e}")
        intentar_estrategia_alternativa(driver)

def cambiar_a_nueva_ventana(driver):
    """Cambia a la ventana más reciente si hay múltiples ventanas."""
    ventanas = driver.window_handles
    if len(ventanas) > 1:
        driver.switch_to.window(ventanas[-1])
        print(f"Cambiado a la ventana: {driver.title}")
    else:
        print("Continuando en la ventana actual.")
    time.sleep(1)

def cambiar_a_iframe_main(driver):
    """Cambia al iframe 'main' intentando varios métodos."""
    try:
        driver.switch_to.frame("main")
        print("Cambiado al iframe 'main'")
        return
    except:
        pass
        
    try:
        iframe = driver.find_element(By.CSS_SELECTOR, "frame[name='main']")
        driver.switch_to.frame(iframe)
        print("Cambiado al iframe usando selector CSS")
        return
    except:
        pass
        
    try:
        frames = driver.find_elements(By.TAG_NAME, "frame")
        if len(frames) > 1:
            driver.switch_to.frame(1) 
            print("Cambiado al iframe usando índice 1")
        else:
            print(f"Se encontraron {len(frames)} frames, pero no se pudo cambiar")
    except Exception as e:
        print(f"No se pudo cambiar al iframe: {e}")
    
    time.sleep(2)

def seleccionar_checkbox_principal(driver):
    """Intenta seleccionar el checkbox 'seleccionar todos'."""
    selectores = [
        '//input[@name="Control"]',
        '//input[@onclick="cambiar()"]',
        '//input[@value="nothing"]',
        '//input[@type="checkbox" and contains(@onclick, "cambiar")]',
        '//td[@align="center"]/input[@type="checkbox"]',
        '//table//tr/td/input[@type="checkbox" and not(@name="lista_ordenes")]'
    ]
    
    for selector in selectores:
        try:
            wait = WebDriverWait(driver, 5)
            checkbox = wait.until(EC.element_to_be_clickable((By.XPATH, selector)))
            time.sleep(random.uniform(1.0, 2.0))
            driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", checkbox)
            time.sleep(random.uniform(0.5, 1.0))
            driver.execute_script("arguments[0].click();", checkbox)
            
            print(f"Checkbox 'seleccionar todos' marcado con selector: {selector}")
            return True
        except:
            continue
    
    print("No se pudo encontrar el checkbox 'seleccionar todos'. Intentando seleccionar individualmente...")
    return False

def intentar_estrategia_alternativa(driver):
    """Intenta una estrategia alternativa usando JavaScript directo."""
    try:
        driver.save_screenshot("error_checkboxes.png")
        print("Captura de pantalla guardada: error_checkboxes.png")
        try:
            driver.switch_to.default_content()
            driver.switch_to.frame("main")
        except:
            pass
        script = """
        var checkboxes = document.querySelectorAll('input[type="checkbox"][name="lista_ordenes"]');
        for (var i = 0; i < checkboxes.length; i++) {
            checkboxes[i].checked = true;
        }
        return checkboxes.length;
        """
        num_seleccionados = driver.execute_script(script)
        print(f"Se seleccionaron {num_seleccionados} checkboxes usando JavaScript directamente")
        driver.switch_to.default_content()
        
    except Exception as e:
        print(f"La estrategia alternativa también falló: {e}")

def generar_listado(driver):

    """
    Hace clic en el botón 'Generar Listado' después de seleccionar los checkboxes.
    """
    try:
        print("Buscando el botón 'Generar Listado'...")
        try:
            driver.switch_to.default_content()
            driver.switch_to.frame("main")
            print("Cambiado al iframe 'main' para buscar el botón")
        except Exception as e:
            print(f"Error al cambiar al iframe: {e}")
        selectores = [
            "//img[contains(@src, 'Generar_Listado') and contains(@onclick, 'ValidarSeleccion')]",
            "//img[@alt='Generar Listado']",
            "//a/img[contains(@src, 'Generar_Listado')]",
            "//a[.//img[contains(@src, 'Generar_Listado')]]"
        ]

        for selector in selectores:
            try:
                
                wait = WebDriverWait(driver, 5)
                boton_generar = wait.until(EC.element_to_be_clickable((By.XPATH, selector)))
                print(f"Botón 'Generar Listado' encontrado con selector: {selector}")
                time.sleep(random.uniform(1.0, 2.0))
                driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", boton_generar)
                time.sleep(random.uniform(0.5, 1.0))
                driver.execute_script("arguments[0].click();", boton_generar)
                print("Se ha hecho clic en el botón 'Generar Listado'")
                time.sleep(5)
                return True
            except Exception as e:
                print(f"No se pudo encontrar el botón con el selector: {selector}")
        
        print("Intentando alternativa con JavaScript...")
        script = """
        var botones = document.querySelectorAll('img');
        for (var i = 0; i < botones.length; i++) {
            if (botones[i].src.includes('Generar_Listado') || 
                (botones[i].alt && botones[i].alt.includes('Generar Listado'))) {
                botones[i].click();
                return true;
            }
        }
        return false;
        """
        resultado = driver.execute_script(script)
        if resultado:
            print("Se ha hecho clic en el botón 'Generar Listado' mediante JavaScript")
            time.sleep(5)
            return True
        else:
            print("No se pudo encontrar el botón 'Generar Listado'")
            return False
            
    except Exception as e:
        print(f"Error al intentar generar el listado: {e}")
        pass
    finally:
        driver.switch_to.default_content()

def seleccionar_formato_excel(driver):
    """
    Selecciona el formato Excel en la ventana de opciones que aparece después de generar el listado.
    """
    try:
        print("Buscando el checkbox de formato Excel...")
        try:
            driver.switch_to.default_content()
            driver.switch_to.frame("main")
            print("Cambiado al iframe 'main' para buscar el checkbox de Excel")
        except Exception as e:
            print(f"Error al cambiar al iframe: {e}")
        
        time.sleep(2)
        selectores = [
            "//input[@name='CSVFormat']",
            "//input[@type='CHECKBOX' and @name='CSVFormat']",
            "//tr[@class='cuasiblanco']//input[@type='CHECKBOX']",
            "//input[following-sibling::text()='Excel']",
            "//td[contains(text(),'Excel')]/input",
            "//td[text()[contains(.,'Excel')]]/input"
        ]
        
        for selector in selectores:
            print("->Revisando cada selector")
            try:
                wait = WebDriverWait(driver, 5)
                checkbox_excel = wait.until(EC.presence_of_element_located((By.XPATH, selector)))
                
                if checkbox_excel.is_selected():
                    print("El checkbox de Excel ya está seleccionado")
                else:
                    time.sleep(random.uniform(0.8, 1.5))
                    driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", checkbox_excel)
                    time.sleep(random.uniform(0.3, 0.7))
                    driver.execute_script("arguments[0].click();", checkbox_excel)
                    print("Checkbox de Excel marcado con el selector: " + selector)
                buscar_y_hacer_clic_en_boton_continuar(driver)
                
                return True
            except Exception as e:
                print(f"No se pudo encontrar el checkbox con el selector: {selector}")

        print("Intentando marcar el checkbox con JavaScript...")
        script = """
        var checkboxes = document.querySelectorAll('input[type="checkbox"]');
        for (var i = 0; i < checkboxes.length; i++) {
            if (checkboxes[i].name === 'CSVFormat' || 
                (checkboxes[i].parentNode && checkboxes[i].parentNode.textContent.includes('Excel'))) {
                if (!checkboxes[i].checked) {
                    checkboxes[i].checked = true;
                    return true;
                } else {
                    return "Ya estaba seleccionado";
                }
            }
        }
        return false;
        """
        resultado = driver.execute_script(script)
        if resultado:
            print(f"Checkbox de Excel marcado mediante JavaScript: {resultado}")
            buscar_y_hacer_clic_en_boton_continuar(driver)
            
            return True
        else:
            print("No se pudo encontrar el checkbox de Excel con JavaScript")
            driver.save_screenshot("buscar_checkbox.png")
            print("Se ha guardado una captura para revisar la estructura: buscar_checkbox.png")
            try:
                html_source = driver.page_source
                print("Primeros 500 caracteres del HTML para diagnóstico:")
                print(html_source[:500])
            except:
                pass
            
            return False
        
    except Exception as e:
        print(f"Error al intentar seleccionar el formato Excel: {e}")

        driver.save_screenshot("error_formato_excel.png")
        print("Se ha guardado una captura de pantalla para diagnóstico: error_formato_excel.png")
        return False
    finally:
        # Volver al contexto principal
        try:
            driver.switch_to.default_content()
        except:
            pass
     
def buscar_y_hacer_clic_en_boton_continuar(driver):
    """
    Busca y hace clic en el botón de continuar después de marcar el checkbox de Excel.
    """
    try:
        print("Buscando el botón de continuar...")
        selectores_boton = [
            "//input[@type='image' and @name='image2']",
            "//input[@type='image' and contains(@src, '1_continuar.gif')]",
            "//input[@type='image' and @id='image2']",
            "//table[@align='CENTER']//input[@type='image']",
            "//img[contains(@src, 'continuar')]",
            "//input[@type='submit' and @value='Continuar']" 
        ]
        
        for selector in selectores_boton:
            try:
                time.sleep(1)
                
                wait = WebDriverWait(driver, 5)
                boton = wait.until(EC.element_to_be_clickable((By.XPATH, selector)))
                time.sleep(random.uniform(0.8, 1.2))
                
                # Hacer clic
                driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", boton)
                time.sleep(random.uniform(0.3, 0.7))
                driver.execute_script("arguments[0].click();", boton)
                
                print(f"Se ha hecho clic en el botón de continuar con selector: {selector}")
                time.sleep(2)
                return True
            except Exception as e:
                print(f"No se pudo hacer clic en el botón con selector: {selector}")
        
        script = """
        var botones = document.querySelectorAll('input[type="image"]');
        for (var i = 0; i < botones.length; i++) {
            if (botones[i].src.includes('continuar') || 
                botones[i].id === 'image2' || 
                botones[i].name === 'image2') {
                botones[i].click();
                return true;
            }
        }
        return false;
        """
        resultado = driver.execute_script(script)
        if resultado:
            print("Se ha hecho clic en el botón de continuar mediante JavaScript")
            time.sleep(2)
            return True
        
        print("Intentando un enfoque alternativo para hacer clic en el botón...")
        try:
            driver.save_screenshot("boton_continuar.png")
            script2 = """
            var forma = document.querySelector('form');
            if (forma) {
                forma.submit();
                return true;
            }
            return false;
            """
            resultado2 = driver.execute_script(script2)
            if resultado2:
                print("Se ha enviado el formulario directamente")
                time.sleep(2)
                return True
            botones_imagen = driver.find_elements(By.TAG_NAME, "input")
            for boton in botones_imagen:
                if boton.get_attribute("type") == "image":
                    from selenium.webdriver.common.action_chains import ActionChains
                    actions = ActionChains(driver)
                    actions.move_to_element(boton).click().perform()
                    print("Se ha hecho clic en el botón usando ActionChains")
                    time.sleep(2)
                    return True
        except Exception as e:
            print(f"Los enfoques alternativos también fallaron: {e}")
        
        print("No se pudo hacer clic en el botón de continuar")
        return False
        
    except Exception as e:
        print(f"Error general al buscar el botón de continuar: {e}")
        return False
    
def bajar_archivo(driver):


    """
    Busca y hace clic en el botón "Bajar Archivo" para descargar el archivo generado.
    """
    try:
        print("Buscando el botón de 'Bajar Archivo'...")
        try:
            driver.switch_to.default_content()
            driver.switch_to.frame("main")
            print("Cambiado al iframe 'main' para buscar el botón de Bajar Archivo")
        except Exception as e:
            print(f"Error al cambiar al iframe: {e}")
        time.sleep(2)
        selectores = [
            "//img[@name='Image6']",
            "//img[contains(@src, 'bajar_archivo.gif')]",
            "//a[contains(@href, 'Download') or contains(@href, '.xls')]//img",
            "//div[@align='center']/a[contains(@href, '.xls')]",
            "//a[contains(@href, '.xls')]",
            "//a[.//img[contains(@src, 'bajar')]]"
        ]
        
        for selector in selectores:
            try:
                wait = WebDriverWait(driver, 5)
                boton = wait.until(EC.element_to_be_clickable((By.XPATH, selector)))
                time.sleep(random.uniform(0.8, 1.5))

                driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", boton)
                time.sleep(random.uniform(0.3, 0.7))
                
                if boton.tag_name.lower() == "img":
                    try:
                        # Buscar el enlace padre
                        enlace = boton.find_element(By.XPATH, "..")
                        if enlace.tag_name.lower() == "a":
                            boton = enlace
                            print("Usando el enlace padre de la imagen para el clic")
                    except:
                        pass
        
                driver.execute_script("arguments[0].click();", boton)
                print(f"Se ha hecho clic en el botón 'Bajar Archivo' usando selector: {selector}")

                time.sleep(2)
                return True
            except Exception as e:
                print(f"No se pudo hacer clic en el botón con selector: {selector}")
        print("Intentando obtener el enlace de descarga directamente...")
        try:
            enlaces = driver.find_elements(By.XPATH, "//a[contains(@href, '.xls') or contains(@href, 'Download')]")
            
            if enlaces:
                url_descarga = enlaces[0].get_attribute("href")
                print(f"URL de descarga encontrada: {url_descarga}")
                driver.execute_script(f"window.open('{url_descarga}', '_blank');")
                print("Abriendo enlace de descarga en nueva pestaña")
                time.sleep(2)
                return True
            else:
                print("No se encontraron enlaces de descarga")
        except Exception as e:
            print(f"Error al intentar obtener enlace directo: {e}")
        
        script = """
        var enlaces = document.querySelectorAll('a');
        for (var i = 0; i < enlaces.length; i++) {
            if (enlaces[i].href.includes('.xls') || 
                (enlaces[i].querySelector('img') && 
                 (enlaces[i].querySelector('img').src.includes('bajar') || 
                  enlaces[i].querySelector('img').name === 'Image6'))) {
                enlaces[i].click();
                return enlaces[i].href;
            }
        }
        return false;
        """
        resultado = driver.execute_script(script)
        if resultado:
            print(f"Se ha hecho clic en el botón mediante JavaScript. URL: {resultado}")
            time.sleep(2)
            return True
        driver.save_screenshot("boton_bajar_archivo.png")
        print("No se pudo encontrar el botón 'Bajar Archivo'. Revisa la captura boton_bajar_archivo.png")
        return False
        
    except Exception as e:
        print(f"Error al intentar bajar el archivo: {e}")
        driver.save_screenshot("error_bajar_archivo.png")
        print("Se ha guardado una captura de pantalla para diagnóstico: error_bajar_archivo.png")
        return False
    finally:
        try:
            driver.switch_to.default_content()
        except:
            pass

def main():
    """Función principal que ejecuta todo el flujo completo."""
    driver = None
    try:
        driver = configurar_navegador()
        url = "https://cencarvajal.com/#/home/welcome"
        driver.get(url)
        time.sleep(5)
        iniciar_sesion(driver, "grandessuperficies@donchicharron.com.co", "Mar1casas@123")
        navegar_a_servicios_transaccionales(driver)
        navegar_a_gestion_orden_compra(driver)
        navegar_a_consulta_orden_compra(driver)
        ir_a_version_anterior(driver)
        print("Esperando a que se cargue la nueva página después de la redirección...")
        time.sleep(8)
        seleccionar_checkboxes(driver, usar_seleccionar_todos=True)
        time.sleep(3)
        generar_listado(driver)
        time.sleep(5)
        seleccionar_formato_excel(driver)
        print("Esperando a que se genere el archivo y aparezca el botón de descarga...")
        time.sleep(8)
        bajar_archivo(driver)
        print("Esperando a que se complete la descarga...")
        time.sleep(10)
        print("Proceso completado exitosamente")
        
        
    except Exception as e:
        print(f"Error general al ejecutar el script: {e}")
        if driver:
            driver.save_screenshot("error_general.png")
            print("Se ha guardado una captura de pantalla para diagnóstico: error_general.png")
    finally:
        pass



In [4]:

# Ejecutar el programa si se ejecuta directamente (no como módulo importado)
if __name__ == "__main__":
    main()

Botón Ingresar clickeado
Servicios transaccionales encontrado y clickeado
Gestión de la Orden de Compra encontrado y clickeado
Consulta de la Orden de Compra encontrado y clickeado
Botón 'Ir a versión del sitio anterior' encontrado y clickeado
Botón 'Continuar al sitio anterior' encontrado y clickeado
Esperando a que se cargue la nueva página después de la redirección...
Cambiado a la ventana: Sistema CEN Fabricante
Cambiado al iframe 'main'
Buscando el botón 'Siguiente'...
Cambiado al iframe 'main' para buscar el botón Siguiente
Usando el enlace padre de la imagen para el clic
Se ha hecho clic en el botón 'Siguiente' usando selector: //img[contains(@src, '2_adelante.gif')]
Buscando checkboxes en la tabla...
Cambiado al iframe 'main'
Checkbox 'seleccionar todos' marcado con selector: //input[@name="Control"]
Volviendo al contexto principal del documento
Buscando el botón 'Generar Listado'...
Cambiado al iframe 'main' para buscar el botón
Botón 'Generar Listado' encontrado con selector:

#### Procesamiento de archivos

In [5]:

def leer_archivos_xls_como_texto(directorio, patron="*.xls", output_file="resultado_combinado.xlsx"):
    """
    Lee archivos con extensión .xls que son realmente archivos de texto tabulado,
    limpia las funciones =T(), y los combina en un único DataFrame.
    
    Args:
        directorio: Ruta al directorio que contiene los archivos
        patron: Patrón para buscar archivos (por defecto "*.xls")
        output_file: Nombre del archivo de salida (por defecto "resultado_combinado.xlsx")
    
    Returns:
        DataFrame combinado
    """
    # Lista todos los archivos que coinciden con el patrón
    archivos = glob.glob(os.path.join(directorio, patron))
    print(f"Archivos encontrados: {len(archivos)}")
    
    dfs = []
    total_filas = 0
    
    # Lista de codificaciones a intentar
    encodings = ['utf-8', 'latin1', 'cp1252']
    
    for archivo in archivos:
        try:
            print(f"Procesando: {os.path.basename(archivo)}")
            
            # Intentar leer con diferentes codificaciones
            contenido = None
            for encoding in encodings:
                try:
                    with open(archivo, 'r', encoding=encoding) as f:
                        contenido = f.read()
                    break
                except UnicodeDecodeError:
                    continue
            
            if contenido is None:
                print(f"  ⚠️ No se pudo leer el archivo. Saltando...")
                continue
            
            # Procesar el contenido
            lineas = contenido.splitlines()
            
            if not lineas:
                print(f"  ⚠️ Archivo vacío. Saltando...")
                continue
            
            # Obtener encabezados de la primera línea
            encabezados = lineas[0].split('\t')
            datos = []
            
            # Procesar líneas de datos
            for linea in lineas[1:]:
                if not linea.strip():
                    continue  # Saltar líneas vacías
                
                valores = linea.split('\t')
                # Asegurar que tenemos el número correcto de valores
                valores = valores[:len(encabezados)]  # Truncar si hay demasiados
                while len(valores) < len(encabezados):
                    valores.append('')  # Rellenar si faltan
                
                # Limpiar funciones =T()
                valores_limpios = [re.sub(r'=T\("([^"]*)"\)', r'\1', v) for v in valores]
                datos.append(valores_limpios)
            
            # Crear DataFrame y añadirlo a la lista
            df = pd.DataFrame(datos, columns=encabezados)
            filas = len(df)
            total_filas += filas
            print(f"  ✓ Leídas {filas} filas y {len(df.columns)} columnas")
            dfs.append(df)
            
        except Exception as e:
            print(f"  ❌ Error: {str(e)}")
    
    # Verificar si se pudieron leer archivos
    if not dfs:
        print("❌ No se pudo leer ningún archivo correctamente.")
        return None
    
    # Concatenar todos los DataFrames
    df_combinado = pd.concat(dfs, ignore_index=True)
    print(f"✅ Combinación completada: {len(df_combinado)} filas totales")
    
    # Guardar el resultado
    df_combinado.to_excel(output_file, index=False)
    print(f"✅ Archivo guardado: {output_file}")
    
    return df_combinado

if __name__ == "__main__":
    directorio = "../../"
    patron = "*.xls"
    archivo_salida = "pedidos_combinados.xlsx"
    
    # Ejecutar la función
    df_resultados = leer_archivos_xls_como_texto(directorio, patron, archivo_salida)
    
    # Mostrar un resumen de los datos (opcional)
    if df_resultados is not None:
        print("\n📊 Resumen de los datos:")
        print(f"- Total de registros: {len(df_resultados)}")
        print(f"- Columnas: {', '.join(df_resultados.columns[:5])}...")
        
        # Mostrar información sobre empresas compradoras
        if 'Empresa Compradora' in df_resultados.columns:
            empresas = df_resultados['Empresa Compradora'].value_counts()
            print("\n📈 Distribución por empresa compradora:")
            for empresa, count in empresas.items():
                print(f"  - {empresa}: {count} pedidos")


Archivos encontrados: 2
Procesando: ORDERS_CO_8829_2025411_131823.xls
  ✓ Leídas 154 filas y 32 columnas
Procesando: ORDERS_CO_8829_2025411_132048.xls
  ✓ Leídas 294 filas y 32 columnas
✅ Combinación completada: 448 filas totales
✅ Archivo guardado: pedidos_combinados.xlsx

📊 Resumen de los datos:
- Total de registros: 448
- Columnas: EAN Empresa Compradora, Nombre Empresa Compradora, Tipo Orden , Número de la Orden, EAN Entidad a Facturar...


#### Proceso de carga de datos 

In [6]:


def upload_pandas_dataframe_to_bigquery(dataframe, project_id, dataset_id, table_id, 
                                       credentials_path, if_exists='replace'):
    """
    Sube un DataFrame de pandas a una tabla de BigQuery.
    
    Args:
        dataframe: DataFrame de pandas con los datos a subir.
        project_id: ID del proyecto de Google Cloud.
        dataset_id: ID del dataset de BigQuery.
        table_id: ID de la tabla de BigQuery.
        if_exists: Acción a tomar si la tabla ya existe ('fail', 'replace', o 'append').
    """
    credentials = service_account.Credentials.from_service_account_file(
        credentials_path,
        scopes=["https://www.googleapis.com/auth/cloud-platform"],
    )
    
    destination_table = f"{project_id}.{dataset_id}.{table_id}"
    dataframe.to_gbq(
        destination_table=destination_table,
        project_id=project_id,
        if_exists=if_exists,
        credentials=credentials
        
    )
    
    print(f"DataFrame subido exitosamente a {destination_table}")
    print(f"Total de filas subidas: {len(dataframe)}")

def limpiar_df_para_bigquery(df):
    """
    Limpia un DataFrame para hacerlo compatible con BigQuery:
    - Normaliza los nombres de columnas (reemplaza espacios y caracteres especiales con _)
    - Asegura que los nombres de columnas sean válidos para BigQuery
    
    Args:
        df: DataFrame de pandas a limpiar
        
    Returns:
        DataFrame limpio con nombres de columnas normalizados
    """
    df_limpio = df.copy()
    columnas_limpias = {}
    for col in df.columns:
        nuevo_nombre = col.strip()
        nuevo_nombre = re.sub(r'[^a-zA-Z0-9]', '_', nuevo_nombre)
        nuevo_nombre = re.sub(r'_+', '_', nuevo_nombre)
        nuevo_nombre = nuevo_nombre.strip('_')
        if nuevo_nombre and nuevo_nombre[0].isdigit():
            nuevo_nombre = 'col_' + nuevo_nombre
            
        if not nuevo_nombre:
            nuevo_nombre = f'columna_{df.columns.get_loc(col)}'
        columnas_limpias[col] = nuevo_nombre
    df_limpio.rename(columns=columnas_limpias, inplace=True)
    print("Nombres de columnas normalizados:")
    for original, nuevo in columnas_limpias.items():
        if original != nuevo:
            print(f"  • '{original}' → '{nuevo}'")
    df_limpio = df_limpio.astype(str)
    return df_limpio

if __name__ == "__main__":
    
    credentials_path = "credentials/croc-454221-e1a3c2e02181.json"
    
    df = limpiar_df_para_bigquery(df_resultados)
    project_id = "croc-454221"
    dataset_id = "db_sales"
    table_id = "tbl_brz_sales_orders_canal_moderno_temp"
    upload_pandas_dataframe_to_bigquery(
        df, 
        project_id, 
        dataset_id, 
        table_id, 
        credentials_path,
        if_exists='append'
    )

Nombres de columnas normalizados:
  • 'EAN Empresa Compradora' → 'EAN_Empresa_Compradora'
  • 'Nombre Empresa Compradora' → 'Nombre_Empresa_Compradora'
  • 'Tipo Orden ' → 'Tipo_Orden'
  • 'Número de la Orden' → 'N_mero_de_la_Orden'
  • 'EAN Entidad a Facturar' → 'EAN_Entidad_a_Facturar'
  • 'Nombre Entidad a Facturar' → 'Nombre_Entidad_a_Facturar'
  • 'EAN Lugar Entrega Factura' → 'EAN_Lugar_Entrega_Factura'
  • 'Nombre Lugar Entrega Factura' → 'Nombre_Lugar_Entrega_Factura'
  • 'F.Documento O/C' → 'F_Documento_O_C'
  • 'F. Mínima Entrega' → 'F_M_nima_Entrega'
  • 'F. Máxima Entrega' → 'F_M_xima_Entrega'
  • 'F.Solicitada de entrega' → 'F_Solicitada_de_entrega'
  • 'Acuerdo Promocional' → 'Acuerdo_Promocional'
  • 'EAN del Ítem' → 'EAN_del_tem'
  • 'Código ítem proveedor' → 'C_digo_tem_proveedor'
  • 'Descripción del ítem' → 'Descripci_n_del_tem'
  • 'EAN Lugar Entrega Ítems' → 'EAN_Lugar_Entrega_tems'
  • 'Nombre Lugar Entrega Ítems' → 'Nombre_Lugar_Entrega_tems'
  • 'Cantidad Total'

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_7640\3764957794.py:19: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  dataframe.to_gbq(


DataFrame subido exitosamente a croc-454221.db_sales.tbl_brz_sales_orders_canal_moderno_temp
Total de filas subidas: 448
